In [1]:
import os
import yaml



In [27]:
# Load in mutation parameters
stream = open("../18_bnbs_rest/test.yaml", 'r')
dictionary = yaml.load(stream)

# Define file paths
template_file = "/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/19_seed_neq/run_all_neq_distrib_wait_template.sh"
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/13/"



/home/zhangi/miniconda3/envs/perses-sims-oct2020/lib/python3.7/site-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
# Define job parameters
wall_time = 1
memory = 5
length = 1
phase = 'complex'

new = list(dictionary.keys())[0]
old, old_aa, new_aa, resid = list(dictionary.values())[0]
job = 89882

In [32]:
# Edit template bash file
with open(template_file, "r") as f:
    lines_out = f.readlines()

lines_new = []
for line in lines_out:
    if "#BSUB -W" in line:
        line = line[:9] + str(wall_time) + line[10:]
    elif "#BSUB -o" in line:
        line = line[:13] + str(new) + f".{phase}.%I.out\n"
    elif "#BSUB -eo" in line:
        line = line[:13] + str(new) + f".{phase}.%I.stderr\n"
    elif "#BSUB -n" in line:
        line = line[:26] + str(memory) + line[27:] 
    elif "#BSUB -J" in line:
        line = line[:13] + str(new) + f'.{phase}.[1-100]"\n'
    elif "#BSUB -w" in line:
        line = line[:9] + f'"done({job})"\n'
    elif "new=" in line:
        line = f"new={new}\n"
    elif "old_aa=" in line:
        line = f"old_aa={old_aa}\n"
    elif "new_aa=" in line:
        line = f"new_aa={new_aa}\n"
    elif "phase=" in line:
        line = f"phase={phase}\n"
    elif "length=" in line:
        line = f"length={length}\n"
    print(line)
    lines_new.append(line)



#!/usr/bin/env bash

# Set walltime limit

#BSUB -W 1:00 

#

# Set output file

#BSUB -o  13.107.complex.%I.out

#

# Set error file

#BSUB -eo 13.107.complex.%I.stderr

#

# Specify node group

#BSUB -m "lu-gpu lv-gpu ld-gpu lt-gpu lp-gpu lg-gpu boson"

#BSUB -q gpuqueue

#

# nodes: number of nodes and GPU request

#BSUB -n 1 -R "rusage[mem=5]"

#BSUB -gpu "num=1:j_exclusive=yes:mode=shared"

#

# job name (default = name of script file)

#BSUB -J "13.107.complex.[1-100]"

# wait for another job to be completed

#BSUB -w "done(89882)"



new=107

old_aa=ASP

new_aa=ALA

phase=complex

length=1



source ~/.bashrc

conda activate perses-sims-oct2020

module load cuda/10.1

python run_neq_distrib_seed_general.py "/data/chodera/zhangi/perses_benchmark/neq/13/$new/" $phase "$((${LSB_JOBINDEX}-1))" $old_aa $new_aa $length # lowercase



In [33]:
# Make dir and save new bash file
os.system(f"mkdir {os.path.join(out_dir, str(new))}")

with open(os.path.join(out_dir, str(new), "run_all_neq_distrib_wait.sh"), "w") as f:
     f.writelines(lines_new)
